In [49]:
import pandas as pd

In [50]:
df_users = pd.read_csv('users.tsv', sep='\t', header=0)
df_hist = pd.read_csv('history.tsv', sep='\t', header=0)
df_val = pd.read_csv('validate.tsv', sep='\t', header=0)
df_val_ans = pd.read_csv('validate_answers.tsv', sep='\t', header=0)

### Удаление дубликатов записей по признаку (1 реклама за 6 часов) (было принято решение сохранить порядок записей в DF)

#### Если пользователь увидел какую-то рекламу, то следующую он смодет увидеть не раньше чем через 6 часов

In [51]:
# Пример данных ДО
df_hist[df_hist['user_id'] == 16085].sort_values('hour').head(30)

,hour,cpm,publisher,user_id
2393,8,157.78,1,16085
25,17,30.00,1,16085
94,17,30.00,1,16085
1125559,17,498.76,1,16085
10105,18,75.00,1,16085
13093,18,80.00,1,16085
1124559,18,40.00,1,16085
12828,22,30.00,1,16085
23411,31,99.12,1,16085
28194,32,43.26,1,16085


In [52]:
# Сохранение исходного порядка строк с помощью индекса
df_hist['original_order'] = range(len(df_hist))

# Временная сортировка для корректного удаления дубликатов
history_sorted = df_hist.sort_values(by=['user_id', 'hour'])

# Вычисление разницы в часах
history_sorted['time_diff'] = history_sorted.groupby('user_id')['hour'].diff()

# Фильтрация данных
df_hist = history_sorted[
    (history_sorted['time_diff'].isna()) | (history_sorted['time_diff'] >= 6)
]

# Восстанавление исходного порядка строк
df_hist = df_hist.sort_values(by='original_order').drop(columns=['original_order', 'time_diff'])

In [53]:
# Пример данных ПОСЛЕ
df_hist[df_hist['user_id'] == 16085].sort_values('hour').head(30)

,hour,cpm,publisher,user_id
2393,8,157.78,1,16085
25,17,30.00,1,16085
23411,31,99.12,1,16085
15835,39,90.00,1,16085
35994,57,162.00,1,16085
53708,79,230.00,1,16085
63051,102,270.00,1,16085
113217,173,80.00,1,16085
198088,297,181.64,1,16085
193875,309,223.92,1,16085


### Замена идентификатора города на "other" при условии, что пользователей из этого города меньше 30 человек

In [54]:
city_counts = df_users['city_id'].value_counts()

threshold = 30  # Порог для "редких городов"
df_users['city_id'] = df_users['city_id'].apply(lambda x: x if city_counts[x] > threshold else 'other')

### Добавление нового признака "CPM в сбалансированных диапазонах"

In [ ]:
df_val['cpm_bin'], bins = pd.qcut(df_val['cpm'], q=5, retbins=True)

# Формирование интервалов
labels = [f'{int(bins[i])}-{int(bins[i+1])}' for i in range(len(bins)-1)]
print("Метки для интервалов:", labels)

df_val['cpm_bin'] = pd.cut(df_val['cpm'], bins=bins, labels=labels, include_lowest=True)
print(df_val['cpm_bin'].value_counts()) 

Метки для интервалов: ['30-67', '67-110', '110-150', '150-250', '250-475']
cpm_bin
67-110     219
30-67      203
250-475    201
110-150    196
150-250    189
Name: count, dtype: int64
